In [6]:
# Rerwite the scrapped walkthroughs

import os
from openai import OpenAI
from dotenv import load_dotenv

# Load environment variables
load_dotenv()

# Set up OpenAI client
client = OpenAI(api_key=os.getenv("OPENAI_API_KEY"))

def rewrite_walkthrough(input_text):
    """
    Rewrites a Pokemon game walkthrough to be more concise using GPT-3.5 Turbo.
    
    :param input_text: The original walkthrough text
    :return: The rewritten, concise walkthrough
    """
    prompt = f"""
    Rewrite the following Pokemon game walkthrough to be more concise and focused on core information.
    Remove unnecessary details and focus on key instructions and important game progress points.
    Keep the essential gameplay elements, item locations, and battle strategies.
    
    Original walkthrough:
    {input_text}
    
    Concise walkthrough:
    """

    try:
        response = client.chat.completions.create(
            model="gpt-4o-mini",
            messages=[
                {"role": "system", "content": "You are a helpful assistant that rewrites Pokemon game walkthroughs to be more concise while retaining key information."},
                {"role": "user", "content": prompt}
            ],
            max_tokens=1000,
            n=1,
            temperature=0.7,
        )
        return response.choices[0].message.content.strip()
    except Exception as e:
        print(f"Error in rewrite_walkthrough: {e}")
        return None

def get_keywords(file_path):
    """
    Extracts keywords from the third line of the file.
    
    :param file_path: Path to the walkthrough file
    :return: Extracted keywords
    """
    try:
        with open(file_path, 'r', encoding='utf-8') as file:
            lines = file.readlines()
            if len(lines) >= 3:
                return lines[2].strip().split(': ')[-1]  # Extract everything after 'Keywords: '
    except Exception as e:
        print(f"Error reading keywords from {file_path}: {e}")
    return "Keywords not found"

def process_walkthrough_files(input_directory, output_directory, game, parts):
    """
    Processes specified walkthrough files for a given game and rewrites them.
    
    :param input_directory: The base directory containing the original walkthrough files
    :param output_directory: The base directory for saving rewritten walkthroughs
    :param game: The specific game folder (e.g., "Black_and_White")
    :param parts: List of part numbers to process
    """
    input_game_directory = os.path.join(input_directory, game)
    output_game_directory = os.path.join(output_directory, game)
    
    # Create output directory if it doesn't exist
    os.makedirs(output_game_directory, exist_ok=True)
    
    for part in parts:
        filename = f"Part {part}.txt"
        input_file_path = os.path.join(input_game_directory, filename)
        
        if os.path.exists(input_file_path):
            keywords = get_keywords(input_file_path)
            print(f"Keywords for Part {part}: {keywords}")
            
            try:
                with open(input_file_path, 'r', encoding='utf-8') as file:
                    original_content = file.read()
                
                print(f"Processing {filename}...")
                rewritten_content = rewrite_walkthrough(original_content)
                
                if rewritten_content:
                    # Create a new filename with game name, part number, and keywords
                    output_filename = f"{game} - Part {part} - {keywords}.txt"
                    output_file_path = os.path.join(output_game_directory, output_filename)
                    
                    with open(output_file_path, 'w', encoding='utf-8') as file:
                        file.write(f"Game: {game}\n")
                        file.write(f"Part: {part}\n")
                        file.write(f"Keywords: {keywords}\n\n")
                        file.write(rewritten_content)
                    
                    print(f"Rewritten walkthrough saved as {output_filename}")
                else:
                    print(f"Failed to rewrite content for {filename}")
            except Exception as e:
                print(f"Error processing {filename}: {e}")
        else:
            print(f"File not found: {input_file_path}")

# # Usage
# input_walkthrough_directory = "walkthroughs"
# output_walkthrough_directory = "walkthrough_rewrites"
# game = "Black_and_White"
# parts_to_process = [1, 2, 3, 4]

# process_walkthrough_files(input_walkthrough_directory, output_walkthrough_directory, game, parts_to_process)

In [8]:
# Rewrite entire folder using above code

import os

def process_all_walkthroughs(input_directory, output_directory):
    """
    Process all walkthrough files for all games in the input directory.
    
    :param input_directory: Base directory containing all game walkthrough folders
    :param output_directory: Base directory for saving rewritten walkthroughs
    """
    for game_folder in os.listdir(input_directory):
        game_path = os.path.join(input_directory, game_folder)
        if os.path.isdir(game_path):
            print(f"Processing game: {game_folder}")
            
            # Get all part numbers for this game
            parts = []
            for filename in os.listdir(game_path):
                if filename.startswith("Part") and filename.endswith(".txt"):
                    part_number = filename.split()[1].split('.')[0]
                    parts.append(int(part_number))
            
            # Sort the parts in ascending order
            parts.sort()
            
            # Process all parts for this game
            process_walkthrough_files(input_directory, output_directory, game_folder, parts)
            
            print(f"Finished processing {game_folder}")

# Usage
input_walkthrough_directory = "walkthroughs"
output_walkthrough_directory = "walkthrough_rewrites"

process_all_walkthroughs(input_walkthrough_directory, output_walkthrough_directory)

Processing game: Black2_and_White2
Keywords for Part 1: Introduction, Aspertia City, Route 19, Floccesy Town
Processing Part 1.txt...
Rewritten walkthrough saved as Black2_and_White2 - Part 1 - Introduction, Aspertia City, Route 19, Floccesy Town.txt
Keywords for Part 2: Route 20, Floccesy Ranch, Pledge Grove, Aspertia Gym
Processing Part 2.txt...
Rewritten walkthrough saved as Black2_and_White2 - Part 2 - Route 20, Floccesy Ranch, Pledge Grove, Aspertia Gym.txt
Keywords for Part 3: Virbank City, Virbank Complex, Virbank Gym, Pokéstar Studios
Processing Part 3.txt...
Rewritten walkthrough saved as Black2_and_White2 - Part 3 - Virbank City, Virbank Complex, Virbank Gym, Pokéstar Studios.txt
Keywords for Part 4: Castelia City, Castelia Sewers, Castelia Gym
Processing Part 4.txt...
Rewritten walkthrough saved as Black2_and_White2 - Part 4 - Castelia City, Castelia Sewers, Castelia Gym.txt
Keywords for Part 5: Route 4, Desert Resort, Relic Castle, Join Avenue
Processing Part 5.txt...
Rewri

In [1]:
import os
from typing import List, Dict
from langchain_community.document_loaders import TextLoader
from langchain.text_splitter import CharacterTextSplitter
from langchain_community.vectorstores import FAISS
from langchain_openai import OpenAIEmbeddings
from langchain.chains import RetrievalQA
from langchain_openai import ChatOpenAI
from dotenv import load_dotenv

load_dotenv()

def load_and_embed_walkthroughs(walkthrough_dir: str = "walkthroughs") -> FAISS:
    documents = []
    for game_folder in os.listdir(walkthrough_dir):
        game_path = os.path.join(walkthrough_dir, game_folder)
        if os.path.isdir(game_path):
            for file_name in os.listdir(game_path):
                file_path = os.path.join(game_path, file_name)
                try:
                    loader = TextLoader(file_path, encoding='utf-8')
                    documents.extend(loader.load())
                except UnicodeDecodeError:
                    try:
                        loader = TextLoader(file_path, encoding='latin-1')
                        documents.extend(loader.load())
                    except Exception as e:
                        print(f"Error loading {file_path}: {e}")

    text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=0)
    texts = text_splitter.split_documents(documents)
    
    embeddings = OpenAIEmbeddings()
    vectorstore = FAISS.from_documents(texts, embeddings)
    
    return vectorstore

# This function can be run in a Jupyter notebook to create the vectorstore
# vectorstore = load_and_embed_walkthroughs()
# vectorstore.save_local("path/to/save/vectorstore")

In [5]:
# pokeapi_tool.py
import requests
from langchain.tools import BaseTool

BASE_URL = "https://pokeapi.co/api/v2/"

class PokeAPITool(BaseTool):
    name = "PokeAPI"
    description = """Use this tool to get information about Pokemon, abilities, moves, items, and more from the PokeAPI.
    Input should be in the format: 'category name_or_id'
    Available categories: pokemon, ability, move, item, type, stat, nature, egg-group, generation, version, location, pokemon-species"""

    def _call(self, query: str) -> str:
        try:
            parts = query.split(maxsplit=1)
            if len(parts) != 2:
                return "Error: Input should be in the format 'category name_or_id'"
            
            category, name_or_id = parts
            category = category.lower()
            name_or_id = name_or_id.lower()

            valid_categories = [
                "pokemon", "ability", "move", "item", "type", "stat", 
                "nature", "egg-group", "generation", "version", "location",
                "pokemon-species"
            ]

            if category not in valid_categories:
                return f"Error: Invalid category. Valid categories are: {', '.join(valid_categories)}"

            response = requests.get(f"{BASE_URL}{category}/{name_or_id}")
            if response.status_code != 200:
                return f"Error: Could not find {name_or_id} in {category}"

            data = response.json()
            
            if category == "pokemon":
                return f"""Name: {data['name']}
                Height: {data['height']}
                Weight: {data['weight']}
                Types: {', '.join([t['type']['name'] for t in data['types']])}
                Abilities: {', '.join([a['ability']['name'] for a in data['abilities']])}
                Base Experience: {data['base_experience']}"""
            elif category == "ability":
                effect = next((e['effect'] for e in data['effect_entries'] if e['language']['name'] == 'en'), "No English description available")
                return f"""Name: {data['name']}
                Effect: {effect}
                Pokemon with this ability: {', '.join([p['pokemon']['name'] for p in data['pokemon']][:5])}"""
            elif category == "move":
                effect = next((e['effect'] for e in data['effect_entries'] if e['language']['name'] == 'en'), "No English description available")
                return f"""Name: {data['name']}
                Type: {data['type']['name']}
                Power: {data['power']}
                Accuracy: {data['accuracy']}
                PP: {data['pp']}
                Effect: {effect}"""
            elif category == "item":
                effect = next((e['effect'] for e in data['effect_entries'] if e['language']['name'] == 'en'), "No English description available")
                return f"""Name: {data['name']}
                Category: {data['category']['name']}
                Effect: {effect}"""
            elif category == "type":
                return f"""Name: {data['name']}
                Double damage to: {', '.join([t['name'] for t in data['damage_relations']['double_damage_to']])}
                Half damage from: {', '.join([t['name'] for t in data['damage_relations']['half_damage_from']])}"""
            elif category == "pokemon-species":
                return f"""Name: {data['name']}
                Generation: {data['generation']['name']}
                Capture rate: {data['capture_rate']}
                Base happiness: {data['base_happiness']}
                Growth rate: {data['growth_rate']['name']}"""
            else:
                return f"Basic info for {category} {name_or_id}: {data}"

        except Exception as e:
            return f"Error: {str(e)}"

    def _arun(self, query: str) -> str:
        # This tool does not support async operations
        raise NotImplementedError("This tool does not support async operations")

In [3]:
# agent_setup.py
from langchain_openai import ChatOpenAI
from langchain.agents import initialize_agent, AgentType
from langchain.memory import ConversationBufferMemory
from pokeapi_tool import PokeAPITool
from dotenv import load_dotenv
import os

load_dotenv()

# Initialize the language model
llm = ChatOpenAI(
    temperature=0,
    model_name="gpt-3.5-turbo",
    openai_api_key=os.getenv("OPENAI_API_KEY")
)

# Initialize the PokeAPI tool
pokeapi_tool = PokeAPITool()

# Initialize conversation memory
memory = ConversationBufferMemory(memory_key="chat_history", return_messages=True)

# Initialize the agent
agent = initialize_agent(
    tools=[pokeapi_tool],
    llm=llm,
    agent=AgentType.CHAT_CONVERSATIONAL_REACT_DESCRIPTION,
    memory=memory,
    verbose=True
)

# Add a system message to give the agent some context
system_message = """You are a helpful Pokemon expert assistant. You can use the PokeAPI tool to retrieve information about Pokemon, abilities, moves, items, types, and more. Always strive to provide accurate and helpful information to the user's questions about Pokemon. If you need specific data, use the PokeAPI tool with the appropriate category and name/id. For example, to get information about Pikachu, you would use the tool with input 'pokemon pikachu'."""

agent.agent.llm_chain.prompt.messages[0].content += system_message

ModuleNotFoundError: No module named 'pokeapi_tool'

In [ ]:
# main.py
from agent_setup import agent

def answer_pokemon_question(question):
    response = agent.run(input=question)
    return response

if __name__ == "__main__":
    print("Welcome to the Pokemon Assistant! Ask me anything about Pokemon.")
    print("Type 'quit' to exit.")
    
    while True:
        user_question = input("\nYour question: ")
        if user_question.lower() == 'quit':
            print("Thank you for using the Pokemon Assistant. Goodbye!")
            break
        
        answer = answer_pokemon_question(user_question)
        print(f"\nAssistant: {answer}")